# Malware Detection in Network Traffic Data

## Введение

Над проектом работают: **Шевцов Владислав**, **Иващенко Дмитрий**.

### Постановка задачи

В данном проекте исследуется датасет [Malware Detection in Network Traffic Data](https://www.kaggle.com/datasets/agungpambudi/network-malware-detection-connection-analysis).

Необходимо построить модель машинного обучения, которая по метаданным каждого сетевого потока классифицирует трафик как "нормальный" и "вредоносный".

### Цели исследования

1. Разведочный анализ и подготовка данных:
	* отобрать ключевой набор признаков
	* обработать пропуски, закодировать категориальные признаки
	* изучить распределения признаков, зависимости и оценить корреляции признаков с целевой переменной
2. Построение и сравнение трёх алгоритмов:
	* логистическая регрессия
	* k-NN
	* XGBoost
3. Оценка качества моделей.
4. Выбор финальной модели и рекомендации.

### О данных

* Каждая строка описывает сетевой поток с более чем 20 атрибутами: IP-адреса и порты, протокол, длительность, счётчики пакетов/байтов, состояние сеанса и др.
* Датасет снабжён двумя уровнями разметки:
	+ `label` — **Benign** или **Malicious** (будет целевой переменной);
	+ `detailed-label` — 10 конкретных подтипов атак (используются лишь для валидации и анализа ошибок, но не для обучения).

## Разведочный анализ данных (EDA)

### Подключение библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.dpi'] = 300
plt.style.use('seaborn-v0_8-whitegrid')

### Загрузка данных

In [ ]:
df = pd.read_csv('dataset/CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv', delimiter='|')

df.shape

### Первичный обзор структуры датасета

In [ ]:
display(df.sample(5, random_state=42))

df.info()

### Предобработка данных

Начнём с того, что сразу же удалим столбцы, которые вряд ли могут пригодиться, а именно:
* `ts` - временная метка события подключения.
* `uid` - уникальный идентификатор соединения.
* `id.orig_h` - 
* `id.resp_h` - 
* `local_orig` и `local_resp` - указывает, считается ли соединение локальным или нет.
* `missed_bytes` - количество пропущенных байтов в соединении.
* `tunnel_parents` - указывает, является ли это соединение частью туннеля.
* `detailed-label` - более подробное описание или метка соединения.

In [ ]:
cols_to_del = [
    'ts',
    'uid',
    'id.orig_h',
    'id.resp_h',
    'local_orig',
    'local_resp',
    'missed_bytes',
    'tunnel_parents',
    'detailed-label'
]

df_copy = df.drop(columns=cols_to_del)

In [ ]:
df_copy.info()

Далее наведём порядок по типу данных для каждого столбца:

In [ ]:
for col in df_copy.columns:
    print(df_copy[col].value_counts())

In [ ]:
df_copy['id.orig_p'] = df_copy['id.orig_p'].astype('int64')

df_copy['id.resp_p'] = df_copy['id.resp_p'].astype('int64')

df_copy['proto'] = df_copy['proto'].astype('string')

df_copy['service'] = df_copy['service'].astype('string')

df_copy['duration'] = df_copy['duration'].astype('string')
df_copy['duration'] = df_copy['duration'].replace('-', '-1')
df_copy['duration'] = df_copy['duration'].astype('float64')

df_copy['orig_bytes'] = df_copy['orig_bytes'].astype('string')
df_copy['orig_bytes'] = df_copy['orig_bytes'].replace('-', '-1')
df_copy['orig_bytes'] = df_copy['orig_bytes'].astype('int64')

df_copy['resp_bytes'] = df_copy['resp_bytes'].astype('string')
df_copy['resp_bytes'] = df_copy['resp_bytes'].replace('-', '-1')
df_copy['resp_bytes'] = df_copy['resp_bytes'].astype('int64')

df_copy['conn_state'] = df_copy['conn_state'].astype('string')

df_copy['history'] = df_copy['history'].astype('string')

df_copy['orig_pkts'] = df_copy['orig_pkts'].astype('int64')

df_copy['orig_ip_bytes'] = df_copy['orig_ip_bytes'].astype('int64')

df_copy['resp_pkts'] = df_copy['resp_pkts'].astype('int64')

df_copy['resp_ip_bytes'] = df_copy['resp_ip_bytes'].astype('int64')

df_copy['label'] = df_copy['label'].astype('string')

Теперь выделим отдельно `label`:

In [ ]:
df_label = df_copy[['label']].copy()

Осталось лишь закодировать `proto`, `service`, `conn_state` и `history`. Будем использовать два подхода: `OrdinalEncoder` и `OneHotEncoder`.

In [ ]:
cat_cols = ['proto', 'service', 'conn_state', 'history']

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

df_enc = df_copy.copy()
df_enc = df_enc.drop(columns=['label'])

print('Before ordinal encoding:')
print(df_enc.values[0])
print(df_enc.shape)

enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

df_enc[cat_cols] = enc.fit_transform(df_enc[cat_cols])

print('After ordinal encoding:')
print(df_enc.values[0])
print(df_enc.shape)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

df_ohe = df_copy.copy()
df_ohe = df_ohe.drop(columns=['label'])

print('Before one-hot encoding:')
print(df_ohe.values[0])
print(df_ohe.shape)

ohe = OneHotEncoder(handle_unknown='ignore',
                    sparse_output=False,
                    dtype='int8',
                    min_frequency=120,
                    drop='first'
      )

encoded = ohe.fit_transform(df_ohe[cat_cols])

new_cols = ohe.get_feature_names_out(cat_cols)

encoded_df = pd.DataFrame(encoded, columns=new_cols, index=df_ohe.index)

df_ohe = pd.concat([df_ohe.drop(columns=cat_cols), encoded_df], axis=1)

print('After one-hot encoding:')
print(df_ohe.values[0])
print(df_ohe.shape)

Итак, теперь мы имеем:
* `df_copy` - предобработанные данные без кодирования `string` данных и со столбцом `label` (это будет удобно для визуализации).
* `df_enc` - предобработанные данные с `OrdinalEncoder`.
* `df_ohe` - предобработанные данные с `OneHotEncoder`.
* `df_label` - метки.

In [ ]:
df_copy.info()

### Анализ данных

#### Распределение целевого признака

In [ ]:
df_label['label'].value_counts().plot(kind='bar', figsize=(6,4))
plt.title('Распределение классов')
plt.xlabel('Класс')
plt.ylabel('Количество')
plt.show()

Отсюда можно сделать вывод, что не придётся бороться с явным дисбалансом классов (хотя с метриками даже при небольшом дисбалансе надо быть аккуратнее). 

#### Распределения категориальных признаков

In [ ]:
cat_cols = ['proto', 'service', 'conn_state', 'history']

TOP_N = 10

count_tables = {}
rate_tables = {}

for col in cat_cols:
    # Берём TOP_N категорий по общему количеству
    top_vals = df_copy[col].value_counts().nlargest(TOP_N).index
    sub = df_copy[df_copy[col].isin(top_vals)]
    
    # Абсолютные частоты
    counts = pd.crosstab(sub[col], sub['label']).reindex(top_vals)
    count_tables[col] = counts
    
    # Доля Malicious
    rates = counts.div(counts.sum(axis=1), axis=0)['Malicious']
    rate_tables[col] = rates

In [ ]:
fig1, axes1 = plt.subplots(2, 2, figsize=(14, 10))
axes1 = axes1.flatten()

for ax, col in zip(axes1, cat_cols):
    counts = count_tables[col]
    x = np.arange(len(counts))
    
    benign = counts['Benign'].values
    malicious = counts['Malicious'].values
    
    ax.bar(x, benign, label='Benign')
    ax.bar(x, malicious, bottom=benign, label='Malicious')
    
    ax.set_title(f'Распределение \"{col}\" Benign/Malicious (Топ {TOP_N})')
    ax.set_xticks(x)
    ax.set_xticklabels(counts.index, rotation=45, ha='right')
    ax.set_ylabel('Количество')
    if col == cat_cols[0]:
        ax.legend()

fig1.tight_layout()

In [ ]:
fig2, axes2 = plt.subplots(2, 2, figsize=(14, 10))
axes2 = axes2.flatten()

for ax, col in zip(axes2, cat_cols):
    rates = rate_tables[col]
    x = np.arange(len(rates))
    
    ax.bar(x, rates.values)
    ax.set_ylim(0, 1)
    ax.set_title(f'\"{col}\" доля Malicious (Топ {TOP_N})')
    ax.set_xticks(x)
    ax.set_xticklabels(rates.index, rotation=45, ha='right')
    ax.set_ylabel('Доля Malicious')

fig2.tight_layout()

In [ ]:
heatmap_df = pd.DataFrame(index=cat_cols, columns=range(TOP_N))

for col in cat_cols:
    rates = rate_tables[col]
    heatmap_df.loc[col, :len(rates)-1] = rates.values

fig3, ax3 = plt.subplots(figsize=(12, 4))
im = ax3.imshow(heatmap_df.astype(float), aspect='auto')

ax3.set_yticks(np.arange(len(cat_cols)))
ax3.set_yticklabels(cat_cols)
ax3.set_xticks(np.arange(TOP_N))
ax3.set_xticklabels([f'Топ {i+1}' for i in range(TOP_N)])
ax3.set_title('Доля Malicious: тепловая карта (Топ категорий)')

cbar = plt.colorbar(im, ax=ax3)
cbar.ax.set_ylabel('Доля Malicious')

fig3.tight_layout()

plt.show()

#### Распределения числовых признаков

In [ ]:
numeric_cols = df_copy.select_dtypes(include=["int64", "float64"]).columns.tolist()

In [ ]:
corr = df_copy[numeric_cols].corr(method="pearson")

plt.figure(figsize=(11, 9))
sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    linewidths=.5,
    cbar_kws={"shrink": .7},
    square=True
)

plt.title("Корреляционная матрица числовых признаков")
plt.show()

In [ ]:
size_sample = 8000
random_state = 42

pair_df = df_copy.sample(n=size_sample, random_state=random_state)

sns.pairplot(
    pair_df,
    vars=numeric_cols,
    hue="label",
)

plt.show()

Первые три признака (порты + duration) практически не связаны с "объёмными" метриками, а последние шесть - это разные способы измерить один и тот же объём трафика.

- `orig_pkts` <-> `orig_ip_bytes`
- `resp_pkts` <-> `resp_ip_bytes`
- `orig_bytes` и `resp_bytes` - почти вложены в IP-байты и тесно связаны с количеством пакетов.

Поэтому такая корреляция вполне естественна.